<a href="https://colab.research.google.com/github/greulist137/Fine-Tuning-Gen-AI-Model/blob/main/LightweightFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique:
* Model:
* Evaluation approach:
* Fine-tuning dataset:

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [ ]:
!pip install huggingface
!pip install datasets
# https://huggingface.co/datasets/dair-ai/emotion

from datasets import load_dataset

dataset = load_dataset("dair-ai/emotion" )

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"]

  Using cached datasets-2.19.1-py3-none-any.whl (542 kB)
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
  Using cached xxhash-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (194 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Let's use a lambda function to tokenize all the examples
# Setting max length to 300 as that is what the max length appears
# to be in the dataset
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=300), batched=True
    )

# Inspect the available columns in the dataset
tokenized_dataset["train"]

In [ ]:
# View the dataset characteristics
dataset["train"]

In [ ]:
dataset["test"]

In [ ]:
dataset["train"][0]

In [ ]:
# Replace <MASK> with the code to unfreeze all the model parameters

from transformers import AutoModelForSequenceClassification
id2Label= {0 :'anger', 1 : 'fear',
    2 : 'joy', 3 : 'love',
    4 : 'sadness', 5 : 'surprise'}
label2Id = {'anger':0, 'fear': 1,
   'joy':2, 'love': 3,
    'sadness': 4, 'surprise': 5}
model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=6,
    id2label=id2Label,
    label2id=label2Id,
)
model.config.pad_token_id = tokenizer.pad_token_id

# Unfreeze all the model parameters.
# Hint: Check the documentation at https://huggingface.co/transformers/v4.2.2/training.html
for param in model.parameters():
    param.requires_grad = True

In [ ]:
!pip install accelerate -U
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/emotion_rating_initial",
        # Set the learning rate
        learning_rate=0.00002,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [ ]:
# Show the performance of the model on the test set
trainer.evaluate()

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [ ]:
!pip install peft
from peft import LoraConfig, TaskType
config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    lora_dropout=0.2
)

In [ ]:
from peft import get_peft_model
lora_model = get_peft_model(model, config)

In [ ]:
lora_model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/emotion_rating_lora_model",
        # Set the learning rate
        learning_rate=0.00002,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [ ]:
lora_model.print_trainable_parameters()

In [ ]:
trainer.train()

In [ ]:
lora_model.save_pretrained('model/lora_model')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
from peft import AutoPeftModelForSequenceClassification
inference_model = AutoPeftModelForSequenceClassification.from_pretrained(
    "model/lora_model",
    num_labels=6,
    id2label=id2Label,
    label2id=label2Id,
)

In [ ]:
inference_model.config.pad_token_id = inference_model.config.eos_token_id

In [ ]:
# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=inference_model,
    args=TrainingArguments(
        output_dir="./data/emotion_rating_lora_model_inference",
        # Set the learning rate
        learning_rate=0.00002,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()